In [ ]:
#ECB cut-and-paste
#importing classes
import random
from random import randint
import os
import string
# The below file is given by OPEN SSL
!pip install pycrypto
from Crypto.Cipher import AES
from math import ceil
from binascii import unhexlify, b2a_base64, a2b_base64, hexlify
import chardet

     |████████████████████████████████| 450kB 5.7MB/s 
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp37-cp37m-linux_x86_64.whl size=499931 sha256=a6d73f73107313f99f5e63f208e4a80847a4da4b7399cc6cabd3919721fb4c32
  Stored in directory: /root/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built pycrypto


In [ ]:
def pkcs7(message, blocksize):
  padlen = blocksize-len(message)%blocksize
  if(padlen == blocksize):
    padlen = 0
  pkcsmessage=bytes([padlen])*padlen
  return message+pkcsmessage

In [ ]:
def random_byte_create(lenofbytes):
  randomnumber=[]
  for i in range(lenofbytes):
    randomnumber.append(random.randint(0,255))
  return bytes(randomnumber)

In [ ]:
Key_global=random_byte_create(16)

In [ ]:
def bytes_to_string(message):
    return "".join(filter(lambda x: x in string.printable, "".join(map(chr, message))))
    
#####~~~~~~~~~~~~~~~~~~~~  END OF HELPER FUNCTION ~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
def parse_cookie_encoding(plaintext):
  #This function takes string input and returns the JSON counterpart
  # split & and = and generate {foo: 'bar',baz: 'qux',zap: 'zazzle'}
  split_plaintext=plaintext.split("&")
  parsed_message={}
  for word in split_plaintext:
    key=word.split("=")[0].strip()
    value=word.split("=")[1].strip()
    parsed_message[key]=value
  return parsed_message


In [ ]:
def parse_cookie_decoding(encodedtext):
  #This function takes JSON as input and returns the cookie
  # Eg: foo=bar&baz=qux&zap=zazzle
  parsed_message=[]
  for word in ["email", "uid", "role"]:
    text="=".join([word, encodedtext[word]])
    parsed_message.append(text)
  return "&".join(parsed_message)


In [ ]:
def profile_for(email_id):
  #Now write a function that encodes a user profile in that format, given an email address.
  #Your "profile_for" function should not allow encoding metacharacters (& and =)
  # Invalid "foo@bar.com&role=admin".
  # Therefore I replace them :)
  return parse_cookie_decoding({
      "email":email_id.replace("&","").replace("=",""),
      "uid": "10",
      "role": "user"
  })

In [ ]:
def encypt_profile_for(user_profile):
  # Encypt AES 
  #ECB Encryption
  plaintext=pkcs7(user_profile,16)
  cipher_object=AES.new(Key_global,AES.MODE_ECB)
  return cipher_object.encrypt(plaintext)

def decypt_profile_for(cookied_profile):
 # AES decryption
  cipher_object=AES.new(Key_global,AES.MODE_ECB)
  plaintext=cipher_object.decrypt(cookied_profile)
  return parse_cookie_encoding(bytes_to_string(plaintext))


In [ ]:
def ecb_cut_and_paste_attacker(block_size=16):
  # Here the ciphertext should themselves generate admin profile
  # Here the attacker should be able to make the role as admin
  #Getting the number of blocks required fore keys like uid, email and role 
  email_user="shris@bar.com"
  #First generate the profile user as convention
  profile_user=profile_for(email_user)
  print("The following message attacker gets from profile for request")
  print(parse_cookie_encoding(profile_user))
  #Encypt the profile user
  encrypted_profile_user=encypt_profile_for(profile_user.encode('utf-8'))
  #Encypt the role--"Admin"
  encypt_admin_only=encypt_profile_for(("admin").encode('utf-8'))
  #The issue is we need a replacement for two bytes "us" therefore I will add 2 more word in email
  # Remove the last block and add admin only block 
  print("The following message attacker gets after modifying the encyption block __ FINAL ANS__")
  return encrypted_profile_user[:-16]+encypt_admin_only


In [ ]:
if __name__== "__main__":
  print(decypt_profile_for(ecb_cut_and_paste_attacker()))

 

The following message attacker gets from profile for request
{'email': 'shris@bar.com', 'uid': '10', 'role': 'user'}
The following message attacker gets after modifying the encyption block __ FINAL ANS__
{'email': 'shris@bar.com', 'uid': '10', 'role': 'admin'}
